<a href="https://colab.research.google.com/github/Ms-Noxolo/Team_EN3_Jozi/blob/master/content_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CONTENT BASED FILTERING

A CONTENT BASED FILTER USES ATTRIBUTES TO RECOMMEND SIMILAR CONTENT.

UNLIKE THE COLLABORATIVE FILTER, THE CONTENT BASED FILTER CAN RECOMMEND A NEW ITEM SO IN THIS CASE IT CAN RECOMMEND A MOVIE FOR THE USER TO WATCH.

###IMPORT LIBRARIES

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In a content-based recommender system, keywords are used to describe the items and a user profile is built to indicate the type of item this user likes. In other words, these algorithms try to recommend items that are similar to those that a user liked in the past.

### IMPORT DATASETS

In [3]:
from google.colab import files
uploaded = files.upload()

Saving imdb_data.csv to imdb_data (1).csv
Saving movies.csv to movies (1).csv


IN ORDER TO BEST RECOMMEND A MOVIE, WE NEED TO LOOK AT THE LIST OF MOVIES WE HAVE.

In [4]:
mov = pd.read_csv('movies.csv')

In [5]:
mov.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


OTHER FEATURES THAT COULD HELP A USER FIND A MOVIE THEY MAY LIKE COULD INVOLVE THE CAST, DIRECTOR AND KEYWORDS

ALL THIS DATA CAN BE FOUND IN THE IMDB DATASET.

In [6]:
imdb = pd.read_csv('imdb_data.csv')

In [7]:
imdb.head()

,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


##COMBINE RELEVANT DATASETS

THE MOVIE DATABASE HAS MOVIE TITLES AND GENRES AND THE IMDB DATASET HAS OUR CAST, DIRECTORS AND KEYWORDS,

WE MAKE IT EASIER TO VIEW ALL THIS INFORMATION BY COMINING THE TWO DATASETS INTO ONE.

In [8]:
alls = pd.merge(mov, imdb)
alls.head()

,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Jumanji (1995),Adventure|Children|Fantasy,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Grumpier Old Men (1995),Comedy|Romance,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Father of the Bride Part II (1995),Comedy,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


###CLEAN DATA
THIS INCLUDES CHECKING FOR NULL VALUES, REMOVING UNNESECCARY CHARACTERS, FILLING BLANK SPACES, ETC

CHECK FOR NAN VALUES

In [9]:
alls.isna().sum()

movieId              0
title                0
genres               0
title_cast        9665
director          9519
runtime          11345
budget           17583
plot_keywords    10482
dtype: int64

In [10]:
alls['genres'] = alls.genres.str.split('|')
alls['title_cast'] = alls['title_cast'].fillna('')
alls['director'] = alls['director'].fillna('')
alls['plot_keywords'] = alls['plot_keywords'].fillna('')
alls.isna().sum()

movieId              0
title                0
genres               0
title_cast           0
director             0
runtime          11345
budget           17583
plot_keywords        0
dtype: int64

In [11]:
alls.head()

,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Father of the Bride Part II (1995),[Comedy],Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


###DROP COLUMNS 

RUNTIME AND BUDGET DO NOT REALLY INFLUENCE A USER'S CHOICE IN MOVIE PREFERNCES SO WE CAN DROP THESE FROM OUR MERGED DATASET. 

In [12]:
alls = alls.drop(['runtime', 'budget'], axis = 1)
alls = alls.drop('movieId', axis = 1)

In [13]:
alls.isna().sum()

title            0
genres           0
title_cast       0
director         0
plot_keywords    0
dtype: int64

WE CONTINUE TO REMOVE ANY EXTRA SYMBOLS THAT COULD MESS WITH OUR RECOMMENDATION SYSTEM.

THE YEARS ARE ALSO REMOVED FROM THE TITLE COLUMN BECAUSE FOR THIS PARTICULAR MODEL, OUR FOCUS IS JUST RETURNING A LIST OF RECOMMENDED MOVIES.

In [14]:
alls['title_cast'] = alls.title_cast.str.split('|')
alls['plot_keywords'] = alls.plot_keywords.str.split('|')
alls['title'] = alls['title'].str.extract('(.*)\((\d{4})\)', expand=False)

In [15]:
alls.head()

,title,genres,title_cast,director,plot_keywords
0,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,"[toy, rivalry, cowboy, cgi animation]"
1,Jumanji,"[Adventure, Children, Fantasy]","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Jonathan Hensleigh,"[board game, adventurer, fight, game]"
2,Grumpier Old Men,"[Comedy, Romance]","[Walter Matthau, Jack Lemmon, Sophia Loren, An...",Mark Steven Johnson,"[boat, lake, neighbor, rivalry]"
3,Waiting to Exhale,"[Comedy, Drama, Romance]","[Whitney Houston, Angela Bassett, Loretta Devi...",Terry McMillan,"[black american, husband wife relationship, be..."
4,Father of the Bride Part II,[Comedy],"[Steve Martin, Diane Keaton, Martin Short, Kim...",Albert Hackett,"[fatherhood, doberman, dog, mansion]"


###WE THEN NEED TO CONVERT THE DATA TO LOWERCASE AND REMOVE SPACES INORDER TO ENSURE THAT OUR SYSTEM DOES NOT CONSIDER MOVIES OR PEOPLE THAT BEGIN WITH THE SAME WORDS AS THE EXACT SAME THING.
E.G "JAMES BOND" AND "JAMES CARTER" COULD BE SEEN AS THE SAME NAME.

In [16]:
def clean(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [17]:
for items in alls:
    alls[items] = alls[items].apply(clean)

In [18]:
alls.head()

,title,genres,title_cast,director,plot_keywords
0,toystory,"[adventure, animation, children, comedy, fantasy]","[tomhanks, timallen, donrickles, jimvarney, wa...",johnlasseter,"[toy, rivalry, cowboy, cgianimation]"
1,jumanji,"[adventure, children, fantasy]","[robinwilliams, jonathanhyde, kirstendunst, br...",jonathanhensleigh,"[boardgame, adventurer, fight, game]"
2,grumpieroldmen,"[comedy, romance]","[waltermatthau, jacklemmon, sophialoren, ann-m...",markstevenjohnson,"[boat, lake, neighbor, rivalry]"
3,waitingtoexhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine,...",terrymcmillan,"[blackamerican, husbandwiferelationship, betra..."
4,fatherofthebridepartii,[comedy],"[stevemartin, dianekeaton, martinshort, kimber...",alberthackett,"[fatherhood, doberman, dog, mansion]"


NOW THAT WE'VE CLEANED ALL OUR DATA AND HAVE DECIDED WHAT FEATURES ARE IMPORTANT, WE CREATE OUR METADATA SOUP THAT WILL BE USED IN VECTORIZING OUR DATA

In [19]:
def soup(x):
    return ' '.join(x['genres']) + ' ' + ' '.join(x['title_cast']) + ' ' + x['director'] + ' ' + ' '.join(x['plot_keywords'])
alls['soup'] = alls.apply(soup, axis=1)

Next we'll write a function that will find us the top n similar papers based on cosine similarity:

## WE THEN RESET THE INDEX

THIS FUNCTION GENERATES A NEW DATAFRAME OR SERIES SETTING THE INDICES IN ORDER, STARTING FROM 0, MAKING IT EASIER TO WORK WITH THE DATAFRAME/SERIES.

In [20]:
alls = alls.reset_index()
indices = pd.Series(alls.index, index=alls['title'])

#COUNTVECTORIZER

IT PROVIDES A SIMPLE WAY TO BOTH TOKENIZER A COLLECTION OF TEXT DOCUMENTS AND BUILD A VOCABULARY OF KNOWN WORDS.

ALSO ENABLING THE PRE-PROCESSING OF TEXT DATA PRIOR TO GENERATING THE VECTOR REPRESENTATION MAKING IT A HIGHLY FLEXIBLE FEATURE REPRESENTATION MODULE FOR TEXT.

In [21]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(alls['soup'])

##COSINE SIMILARITY

A METRIC USED TO DETERMINE HOW SIMILAR DATA/DOCUMENTS ARE.
IT MEASURES SIMILARITY BETWEEN TWO TWO NON-ZERO VECTORS OF AN INNER PRODUCT SPACE THAT MEASURES THE COSINE OF THE ANGLE BETWEEN THEM.

In [22]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

THE FINAL STEP BEFORE ASKING A RECOMMENDATION WILL BE TO WRITE A FUNCTION THAT CONSIDERS ALL OUR RELEVANT FEATURES ALONG WITH THE CONSINE SIMILARITY AND THEN RETRUNS A LIST OF RECOMENDATIONS.

In [23]:
def reco(title, cosine_sim = cosine_sim):
    
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return alls['title'].iloc[movie_indices]

FINALY WE ARE NOW ABLE TO SUGGEST A MOVIE AND FIND OTHER SIMILAR MOVIES FOR OUR USER TO WATCH.  

In [24]:
reco('jumanji', cosine_sim)

986                           escapetowitchmountain
1953                  darbyo'gillandthelittlepeople
2002                                     returntooz
2070                           neverendingstory,the
2307                            santaclaus:themovie
10715    brotherslionheart,the(brödernalejonhjärta)
15233                             aliceinwonderland
24008                   journeytothebeginningoftime
898                                  wizardofoz,the
1943                              blackcauldron,the
Name: title, dtype: object